In [ ]:
"""
NOTES:
    - DraftsActive will be available as soon as the draft is entered regardless
    of fill status.
    - DraftDetail picks 

TODO:
    - A shell of the draft will need to be created at the beginning in order
    to derive the # of picks between selections for each user/pick
        - Add a 'users' and 'draft_entries' df to DraftDetail which pulls the 
        data found in the url_draft url and sliced using ['draft']['users']
        and ['draft']['draft_entries']
        - Will want to wait until the status (i.e. ['draft']['status'] == 'drafting')
        - Need to update DraftsDetail to account for IndexError that occurs
        when creating the main df at the very beginning of the draft when
        'picks' is empty


"""

In [2]:
from os.path import join
import pickle

import pandas as pd
import numpy as np
import getpass

import UD_draft_model.scrapers.scrape_site.scrape_league_data as scrape_site
import UD_draft_model.scrapers.scrape_site.pull_bearer_token as pb
import UD_draft_model.data_processing.prepare_drafts as prepare_drafts
import UD_draft_model.data_processing.add_features as add_features
from UD_draft_model.modeling.model_version import ModelVersion

In [3]:
import getpass


pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

### Variables to change ###
chromedriver_path = "/usr/bin/chromedriver"
username = input("Enter Underdog username: ")
password = getpass.getpass()

### Keep as is ###
url = "https://underdogfantasy.com/lobby"
bearer_token = pb.pull_bearer_token(url, chromedriver_path, username, password)

In [18]:
import os
import json


def save_bearer_token(username: str, bearer_token: str) -> None:
    """
    Saves the bearer token to a json file to prevent re-scraping the token
    when it's still active
    """

    token_data = {user: token}

    token_path = create_token_path()
    


user = 'condelong11@yahoo2.com'
token = 'abc'

token_data = {user: token}



with open(token_path, 'w') as f:
    json.dump(token_data, f)



In [35]:
# def create_token_path() -> str:
#     """ 
#     Creates the path to the bearer_token data.
#     """

#     cwd = os.getcwd()
#     folder_end_index = cwd.rfind('UD_draft_model') + len('UD_draft_model')

#     token_dir = os.path.join(cwd[:folder_end_index], 'bearer_token')
#     token_path = os.path.join(token_dir, 'bearer_token.json')

#     return token_path


def create_token_path() -> str:
    """ 
    Creates the path to the bearer_token data.
    """

    relative_path = 'scrapers/scrape_site/bearer_token'

    cwd = os.getcwd()
    folder_end_index = cwd.rfind('UD_draft_model') + len('UD_draft_model')

    if cwd.rfind('UD_draft_model') == -1:
        base_path = os.path.join(cwd, 'UD_draft_model')
    else:
        base_path = cwd[:folder_end_index]

    token_dir = os.path.join(base_path, relative_path)
    token_path = os.path.join(token_dir, 'bearer_token.json')

    return token_path


def read_bearer_token(file_path: str=None) -> dict:
    """ 
    Reads in all user's bearer tokens.
    """

    if file_path is None:
        file_path = create_token_path()

    with open(file_path, 'r') as f:
        json_data = json.load(f)

    return json_data


def save_bearer_token(username: str, bearer_token: str) -> None:
    """
    Saves the bearer token to a json file to prevent re-scraping the token
    when it's still active.
    """

    token_path = create_token_path()

    try:
        token_data = read_bearer_token(file_path=token_path)
    except FileNotFoundError:
        token_data = {}

    token_data[username] = bearer_token
    print(token_data)

    with open(token_path, 'w') as f:
        json.dump(token_data, f)

    return None

check = create_token_path()
# json_data = read_bearer_token()

save_bearer_token(username, 'aaa')


{'condelong11@yahoo.com': 'aaa'}


In [3]:
player_vars = [
    'appearance_id', 'player_id', 'position', 'first_name', 'last_name',
    'abbr', 'team_name', 'adp', 'season_projected_points'
]

active_drafts = scrape_site.DraftsActive(bearer_token)

df_active = active_drafts.create_df_active_drafts()

draft_entry_id = df_active['draft_entry_id'].iloc[0]
slate_id = df_active['slate_id'].iloc[0]
scoring_type_id = df_active['scoring_type_id'].iloc[0]
draft_id = [df_active['id'].iloc[0]]
rounds = df_active['rounds'].iloc[0]
refs = scrape_site.ReferenceData(slate_id, scoring_type_id)

draft_detail = scrape_site.DraftsDetail(draft_id, bearer_token)

df_players = refs.create_df_players_master()
df_players = df_players[player_vars]
df_draft = draft_detail.create_df_drafts()
df_entries = draft_detail.create_df_draft_entries()

# url = [url for url in draft_detail.url_drafts.values()][0]
# header = draft_detail.auth_header
# header['authorization'] = bearer_token
# _df = draft_detail.read_in_site_data(url, headers=header)


# df
df_players
df_active

,id,auto_pick_at,clock,contest_style_id,draft_at,draft_entry_id,draft_type,entry_count,entry_role,entry_style_id,pick_count,slate_id,source,source_entry_style_id,status,title,user_auto_pick,user_pick_order,scoring_type_id,rounds
0,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,2023-03-18T18:30:25Z,28800,978b95dd-7c25-467c-83c9-332d90a557a4,2023-03-16T17:27:23Z,6667f354-e6b1-49ed-842f-7e029d5f3920,slow,12,None,473af08f-55a4-5f56-b401-2e6ce8d0e096,21,b84244dc-aa63-4b62-bdd5-8fccd365c074,sit_and_go,None,drafting,None,off,5,ccf300b0-9197-5951-bd96-cba84ad71e86,20


In [20]:
def create_draft_board(df_entries: pd.DataFrame, num_rounds: int) -> pd.DataFrame:
    """
    Creates a df of all user/pick numbers given the entries of a snake draft.

    Parameters
    ----------
    df_entries : pd.DataFrame
        All entries in the draft.
        Note that the draft must be full
    num_rounds : int
        Number of rounds in the draft.

    Returns
    -------
    pd.DataFrame
        Draft board containing each pick number for every entry.
    """

    df_asc = df_entries.sort_values(by='pick_order')
    df_desc = df_entries.sort_values(by='pick_order', ascending=False)

    dfs = []
    for round in range(1, num_rounds + 1):
        if round % 2 == 0:
            df_round = df_desc.copy()
        else:
            df_round = df_asc.copy()

        df_round['round'] = round

        dfs.append(df_round)

    df = pd.concat(dfs).reset_index(drop=True)
    df['number'] = df.index + 1
    df.rename(columns={'id': 'draft_entry_id'}, inplace=True)

    keep_vars = [
        'draft_id', 'draft_entry_id', 'user_id', 'username', 'round', 'number'
    ]

    df = df[keep_vars]

    return df


def update_board(df_board: pd.DataFrame, df_draft: pd.DataFrame) -> pd.DataFrame:
    """
    Updates the draft board with the selections made.

    Parameters
    ----------
    df_board : pd.DataFrame
        Draft board containing each pick number for every entry.
    df_draft : pd.DataFrame
        All draft selections that have been made so far.

    Returns
    -------
    pd.DataFrame
        Draft board that contains the player selection made at each pick.
    """

    df = pd.merge(
        df_board, df_draft, 
        how='left', on=['draft_id', 'draft_entry_id', 'number']
    )

    return df


def add_user_next_pick_number(df_board: pd.DataFrame, draft_entry_id: str
) -> pd.DataFrame:
    """
    Adds the pick number of the next pick of interest for the user using
    the app rather than the user currently drafting. "Next pick of interest" 
    refers to the pick number that follows the user's upcoming pick.

    next_pick_number is used to create features that indicate how each players'
    rank relates to the # of picks between the current pick and the 
    next pick of the user currently drafting. Therefore, probability estimates
    will be from the perspective of this user which isn't relevant to the user
    actually utilizing the model. For example, if the user is drafting from pick 6
    and the draft is currently at the 10th pick, probability estimates for pick
    15 are useless. Instead, estimates for pick 30 would allow the user to 
    start preparing for their next pick.

    IMPORTANT: Using this will result in feature and target distributions
    being different from training (e.g. max picks between will be 48 vs. 24)
    which could throw the predictions off. However, this should be insignificant
    once the draft is within a handful of picks from the user.

    Parameters
    ----------
    df_board : pd.DataFrame
        Draft board containing each pick number for every entry.
    draft_entry_id : str
        Draft entry ID to base the next pick number off of.

    Returns
    -------
    pd.DataFrame
        Draft board with the next pick number based off the draft_entry_id
        passed.
    """

    df_user = df_board.loc[df_board['draft_entry_id'] == draft_entry_id].copy()
    df_user.sort_values(by='number', inplace=True)

    df_user['next_pick_number_2'] = df_user['next_pick_number'].shift(-1)

    rename_vars = {
        'number': 'user_number',
        'next_pick_number': 'user_next_pick_number',
        'next_pick_number_2': 'user_next_pick_number_2'
    }
    keep_vars = ['round', 'number', 'next_pick_number', 'next_pick_number_2']
    df_user = df_user[keep_vars].rename(columns=rename_vars)

    # User's next pick number needs to be named 'next_pick_number' for the
    # add_features function.
    df_all = df_board.rename(columns={'next_pick_number': 'next_pick_number_og'})

    df = pd.merge(df_all, df_user, how='left', on='round')
    df['next_pick_number'] = np.where(
        df['number'] <= df['user_number'], 
        df['user_next_pick_number'],
        df['user_next_pick_number_2']
    )

    df.drop(columns=list(rename_vars.values()), inplace=True)

    return df


def get_current_pick(df_board: pd.DataFrame) -> pd.DataFrame:
    """
    Filters the draft board down to the current pick.

    Parameters
    ----------
    df_board : pd.DataFrame
        Draft board that includes all current selections.

    Returns
    -------
    pd.DataFrame
        Filtered Draft board that only contains the next pick to select
        a player.
    """

    df = df_board.loc[df_board['appearance_id'].isnull()].iloc[0:1]

    return df


def get_avail_players(df_players: pd.DataFrame, df_draft: pd.DataFrame) -> pd.DataFrame:
    """
    Pulls the remaining available players to draft.

    Parameters
    ----------
    df_players : pd.DataFrame
        All players that could/can be selected in the draft.
    df_draft : pd.DataFrame
        All draft selections that have been made so far.

    Returns
    -------
    pd.DataFrame
        All players that have NOT been drafted.
    """

    df = (
        df_players
        .loc[~df_players['appearance_id'].isin(df_draft['appearance_id'])]
        # .iloc[:100]
    )

    return df


def add_avail_players(df_cur_pick: pd.DataFrame, df_avail_players: pd.DataFrame
) -> pd.DataFrame:
    """
    Joins the available players onto the current pick df that is required
    for the model features and prediction.

    Parameters
    ----------
    df_cur_pick : pd.DataFrame
        Filtered Draft board that only contains the next pick to select
        a player.
    df_avail_players : pd.DataFrame
        All players that have NOT been drafted.

    Returns
    -------
    pd.DataFrame
        All players that have NOT been drafted with current pick data.
    """

    left_vars = [
        'draft_id', 'draft_entry_id', 'user_id', 'username', 'round',
        'number', 'next_pick_number'
    ]
    df = pd.merge(df_cur_pick[left_vars], df_avail_players, how='cross')
    df.reset_index(inplace=True, drop=True)

    return df


def load_model(file_path: str) -> ModelVersion:
    """
    Loads the ModelVersion object containing the model to be used for creating
    predictions.

    Parameters
    ----------
    file_path : str
        Path to the ModelVersion object

    Returns
    -------
    ModelVersion
        ModelVersion object with model to be used.
    """
    dbfile = open(file_path, 'rb')
    obj = pickle.load(dbfile)
    dbfile.close()

    return obj


def create_predictions(df: pd.DataFrame, model: ModelVersion) -> np.ndarray:
    """
    Applies the model to the df to create new predictions.

    Parameters
    ----------
    df : pd.DataFrame
        df of all players to create a probability estimate for.
    model : ModelVersion
        ModelVersion obj that contains the model to use for predictions.

    Returns
    -------
    np.ndarray
        1D array containing the probability estimates.
    """

    df_features = df[model.metadata['features']]

    y_pred_prob = model.model.predict_proba(df_features)
    y_pred_prob = y_pred_prob[:,1]

    return y_pred_prob


def merge_prediction(df: pd.DataFrame, pred: np.array, out_col: str) -> pd.DataFrame:
    """
    Merges the model's predicions back onto the full df.

    Parameters
    ----------
    df : pd.DataFrame
        df of all available players to draft.
    pred : np.array
        1D array of probability estimates.
    out_col : str
        Name of the probability estiamte column.

    Returns
    -------
    pd.DataFrame
        df of all available players to draft with the probability estimate
        of being picked added.
    """

    df = df.copy()
    df.reset_index(inplace=True, drop=True)

    pred = pd.DataFrame(pred, columns=[out_col])

    df = pd.concat([df, pred], axis=1)
    df[out_col] = df[out_col].astype(float).round(2)

    return df


MODEL_PATH = (
    "/home/cdelong/Python-Projects/UD-Draft-Model/"
    + "Repo-Work/UD-Draft-Model/data/models"
)
MODEL = "LogisticRegression_v01_v001"

model = load_model(join(MODEL_PATH, MODEL))

df_board = create_draft_board(df_entries, rounds)
df_board = prepare_drafts.add_next_pick_number(df_board, filter_nulls=False)
df_board = add_user_next_pick_number(df_board, draft_entry_id)

df_board = update_board(df_board, df_draft)
df_cur_pick = get_current_pick(df_board)
df_avail_players = get_avail_players(df_players, df_draft)
df_w_players = add_avail_players(df_cur_pick, df_avail_players)

df_w_features = add_features.add_features(df_w_players)

probs = create_predictions(df_w_features, model)
df_w_probs = merge_prediction(df_w_features, probs, 'prob')

df_w_probs

,draft_id,draft_entry_id,user_id,username,round,number,next_pick_number,appearance_id,player_id,position,first_name,last_name,abbr,team_name,adp,season_projected_points,avail_cur_rank_actual,picks_btwn,diff_cur_rank_picks_btwn,ind_rank_btwn,prob
0,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,7293e9c2-dc9c-4cf6-9a4f-d38a4c5835a6,f1766f00-503b-4b13-a5d1-cd4347423121,QB,Patrick,Mahomes,KC,Kansas City Chiefs,19.0,386.9,1,22.0,-21.0,1,0.97
1,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,e1fce0bc-3941-43ad-bb7a-9f7ed177dcf4,3bbedd10-bec5-49ae-976b-659119ab156c,QB,Jalen,Hurts,PHI,Philadelphia Eagles,20.6,357.1,2,22.0,-20.0,1,0.97
2,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,78cffe01-8a46-4751-a4c6-cd2b0c5dc910,9e8bc8ee-72ed-4792-acca-d2b744a0b5ba,WR,DeVonta,Smith,PHI,Philadelphia Eagles,22.3,197.7,3,22.0,-19.0,1,0.96
3,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,97ee0915-b241-463d-b6dd-b45589ad6caa,463ed573-6658-435f-82a4-170352fb6887,WR,Chris,Olave,NO,New Orleans Saints,24.2,193.8,4,22.0,-18.0,1,0.96
4,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,10341d1f-04bf-4231-aa4a-8e8496cb7f22,f35e2ebd-3564-4083-b64a-0d59bedf1d9c,RB,Breece,Hall,NYJ,NY Jets,25.2,212.2,5,22.0,-17.0,1,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,774e0c10-3b95-4c28-8c07-b41cea33a63a,5a8dfec6-a864-49ff-b02e-8c64192bce48,TE,Jace,Sternberger,NaN,NaN,-,0.0,1539,22.0,1517.0,0,0.00
1539,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,7759e140-b154-40c9-9f72-b13efafeea5f,3fcdf736-9272-4e4f-bbf7-57269a23da38,RB,Xavier,Jones,NaN,NaN,-,0.0,1540,22.0,1518.0,0,0.00
1540,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,777b7808-b98a-4a72-80e8-6a08e8dc7916,4249eca7-1da3-43d8-ba3c-8aac0f4157f6,WR,Ryan,McDaniel,NaN,NaN,-,0.0,1541,22.0,1519.0,0,0.00
1541,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,2,22,44.0,7789e2a7-8244-4cc7-b523-319691f06153,685aaeb8-9d0b-4823-92cb-bff8cec67076,TE,JJ,Howland,NaN,NaN,-,0.0,1542,22.0,1520.0,0,0.00


In [5]:
pd.set_option("display.max_rows", 300)

df_board

,draft_id,draft_entry_id,user_id,username,round,number,next_pick_number_og,next_pick_number,id,appearance_id,created_at,pick_slot_id,points,projection_adp,projection_points,swapped
0,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,db1d2319-bd54-4708-845a-0000724b79dc,c062476c-e168-4a6f-b050-fc623fa4a8f2,MATTYG019,1,1,24.0,20.0,f79643ea-a65c-4f9c-b817-72c9ef8e68eb,0e383613-0c4f-4f3c-8145-73c0863a1929,2023-03-16T17:31:28Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,1.2,288.7,False
1,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,87b6f306-b701-4c93-bba6-ba9b88869a98,4053a958-5980-47a3-b2d7-5d175c019579,BASEDTRENT,1,2,23.0,20.0,b7dbc2bb-9b6a-4957-9c12-836dcfb96b4c,0a57b8db-0a17-4b0b-bab2-cfadb788d9b8,2023-03-16T17:31:48Z,8b25d7d2-613b-4a5a-b47b-a6ea125841c0,0.00,3.0,303.5,False
2,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,e33d1134-6dfc-4aea-b8f8-1965c01ee6bb,42246640-ad60-4e58-b3b2-8b70352e5a9f,LEWASH,1,3,22.0,20.0,b72d25dd-9fcd-49d2-b787-9b02e58ae82b,c307679c-8ee9-47e3-882f-f386bc00f510,2023-03-16T18:10:07Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,2.4,271.7,False
3,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,613e8191-1841-4b09-ba02-cc3f62b467d8,2be67e8d-e99c-4b6e-af71-2e0551a57c56,MFL10OLDHEAD,1,4,21.0,20.0,71b918f8-14a7-48bf-9948-c387a7d5a38e,1d7bfbe8-c4db-4984-a644-d30d1e07e25f,2023-03-16T18:15:16Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,7.3,283.6,False
4,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,6667f354-e6b1-49ed-842f-7e029d5f3920,03717131-e053-451d-9be6-09755f5875ae,CONNORDELONG,1,5,20.0,20.0,75605430-eb14-459c-a4a7-12b7809ad07b,e32a0878-ba15-4fb1-8518-d86663bbaa17,2023-03-16T18:19:03Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,5.0,302.0,False
5,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,85890d79-ed5b-4f6f-9c70-e894eaca4bc9,f972412e-3e67-4113-9a66-11451ff4a09c,CORNHUSKINGHOG,1,6,19.0,29.0,26c0a053-0586-4579-b7ca-511a70bb3bd7,c7bc600e-e3be-48ff-86cf-3b1d0ad42a5f,2023-03-17T00:30:39Z,16a20025-458b-4d97-a2cf-4c6516e040d1,0.00,5.1,209.2,False
6,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,1,7,18.0,29.0,64ace2ec-a995-4c0c-8ac9-0f00b670672a,8db7e427-3f92-499e-a538-b585effeb21c,2023-03-17T00:41:35Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,5.5,256.2,False
7,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,af93693e-cc98-4ab6-95bd-7357e05788b3,94f988d3-6771-4e7d-b497-2b10a7d47881,CMFARRELLPGA,1,8,17.0,29.0,caa23b57-43ef-4ed1-8c2a-c79513a10a07,fdb55eb7-fb59-4d56-83fa-cd8b6df1b279,2023-03-17T00:57:15Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,8.7,230.3,False
8,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,6aebe929-34dd-4993-bee3-0148a5e71061,6cfccec1-e710-416a-90dd-9e15c705b730,FOOTBALLSMITTY8,1,9,16.0,29.0,f0c6c215-25da-473d-9026-b08bf4bf26a8,fc2b0611-c973-41a1-ab52-670d2bd63fbe,2023-03-17T01:00:26Z,8b25d7d2-613b-4a5a-b47b-a6ea125841c0,0.00,11.3,270.7,False
9,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,d9dd5f64-cc5a-4c27-8d95-166751761a3a,884a2d11-08e4-4a11-b159-9603c4845016,JTSPATRIOTS38,1,10,15.0,29.0,95312a27-7b89-4433-9260-0fe93bceace7,f1bab874-27df-4367-939b-8a0d1a87d5fe,2023-03-17T01:19:33Z,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,9.9,245.2,False


In [17]:
def create_predictions(df: pd.DataFrame, model: ModelVersion) -> np.ndarray:
    """
    Applies the model to the df to create new predictions.

    Parameters
    ----------
    df : pd.DataFrame
        df of all players to create a probability estimate for.
    model : ModelVersion
        ModelVersion obj that contains the model to use for predictions.

    Returns
    -------
    np.ndarray
        1D array containing the probability estimates.
    """

    df_features = df[model.metadata['features']]

    y_pred_prob = model.model.predict_proba(df_features)
    y_pred_prob = y_pred_prob[:,1]

    return y_pred_prob


def merge_prediction(df: pd.DataFrame, pred: np.array, out_col: str) -> pd.DataFrame:
    """
    Merges the model's predicions back onto the full df.

    Parameters
    ----------
    df : pd.DataFrame
        df of all available players to draft.
    pred : np.array
        1D array of probability estimates.
    out_col : str
        Name of the probability estiamte column.

    Returns
    -------
    pd.DataFrame
        df of all available players to draft with the probability estimate
        of being picked added.
    """

    df = df.copy()
    df.reset_index(inplace=True, drop=True)

    pred = pd.DataFrame(pred, columns=[out_col])

    df = pd.concat([df, pred], axis=1)
    df[out_col] = df[out_col].astype(float).round(2)

    return df

# _df = df_w_features[model.metadata['features']]

# y_pred_prob = model.model.predict_proba(_df)
# y_pred_prob = y_pred_prob[:,1]

probs = create_predictions(df_w_features, model)
df = merge_prediction(df_w_features, probs, 'prob')

probs.ndim

1

In [38]:
out = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/UD_draft_model/scratch'
active_drafts = scrape_site.DraftsActive(bearer_token)

df = pd.read_pickle(join(out, 'active_drafts2.pkl'))
df = active_drafts._add_scoring_type(df.copy())

slate_id = df['slate_id'].iloc[0]
scoring_type_id = df['scoring_type_id'].iloc[0]
draft_id = [df['id'].iloc[0]]
refs = scrape_site.ReferenceData(slate_id, scoring_type_id)

draft_detail = scrape_site.DraftsDetail(draft_id, bearer_token)

df_players = refs.create_df_players_master()
df_draft = draft_detail.create_df_drafts()